In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install torch==1.5.0
!pip install transformers

#Hate Speech Detection dataset, available through TweetEval
!git clone https://github.com/cardiffnlp/tweeteval /tmp/tweeteval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 752.0 MB 10 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.5.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.5.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.5.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 34.5 MB/s 
     |████████████████████████████████| 86 kB 3.5 MB/s 
    

In [3]:
from transformers import pipeline

**Predict what the < mask > is hiding:**
# **"Send these < mask > back!"**

In [4]:
#Roberta base prediction with pipeline

fill_mask = pipeline(
    "fill-mask",
    model="roberta-base",
    tokenizer="roberta-base"
)
fill_mask("Send these <mask> back!")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[{'score': 0.16661563515663147,
  'sequence': 'Send these pictures back!',
  'token': 3493,
  'token_str': ' pictures'},
 {'score': 0.10792822390794754,
  'sequence': 'Send these photos back!',
  'token': 2356,
  'token_str': ' photos'},
 {'score': 0.07670954614877701,
  'sequence': 'Send these emails back!',
  'token': 5575,
  'token_str': ' emails'},
 {'score': 0.04860773682594299,
  'sequence': 'Send these images back!',
  'token': 3156,
  'token_str': ' images'},
 {'score': 0.048417724668979645,
  'sequence': 'Send these letters back!',
  'token': 5430,
  'token_str': ' letters'}]

In [5]:
from transformers import RobertaTokenizer, RobertaForMaskedLM

#base Roberta tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [6]:
#import dataset

from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/tmp/tweeteval/datasets/hate/train_text.txt",
    block_size=512,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [7]:
#hate dataset overview (quite rude)

f = open("/tmp/tweeteval/datasets/hate/train_text.txt", 'r')
lines = f.readlines()
lines = lines[28:52]
print(lines)

['Federal judge to #Trump: "imminent danger" of "irreparable injury to #refugees, visa-holders", etc. @user \n', "This bitch whom I won't name on Facebook is such a shameless woman ruining a relationship. #getalife #stopspreadinglies #hoe \n", "@user @user Omg he gave her his number and she didn't say no what a cunt The girl is the cunt for giving it to some random fucktard \n", 'Why we need to protect refugees from the ‘˜big ideas’ designed to save them @user @user @user @user @user @user @user \n', 'Kevin Rosero of the Latino Immigrant Experience Oral History Project #NJHRIC #NewarkPublicLibrary needs help identifying Newark Latino immigrant residents to interview and capture their stories and experiences. MORE INFO: \n', '@user Are you pussy grabbing tonight?? \n', "Some people really dont understand why I get so upset at injustices happening in this country. Trust me, it's not that I like getting my panties in a wad over dumb things. It's the fact that people like me (minority, bro

In [8]:
#data collator form batches of same length, processes and apply data augmentation

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
#TrainingArguments holds fields for the training process
#Trainer brings all the objects together

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-retrained",
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=48,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [10]:
#Train
#trainer.train()

In [11]:
#trainer.save_model("/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL/Presentation/roberta-retrained")

In [12]:
#from transformers import AutoModel

#model = AutoModel.from_pretrained("/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL/Presentation/roberta-retrained")

In [13]:
#Import saved model trained with Hate Dataset and do new prediction

from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model= "/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL/Presentation/roberta-retrained",
    tokenizer="roberta-base"
)


fill_mask("Send these <mask> back!")

loading configuration file /content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL/Presentation/roberta-retrained/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/Othercomputers/Mi porta\u0301til/Master/GitHub/twsm-PERSONAL/Presentation/roberta-retrained",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.19.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file /content/drive/Othercomputers/Mi porta

[{'score': 0.1508449912071228,
  'sequence': 'Send these people back!',
  'token': 82,
  'token_str': ' people'},
 {'score': 0.09431503713130951,
  'sequence': 'Send these kids back!',
  'token': 1159,
  'token_str': ' kids'},
 {'score': 0.0786794126033783,
  'sequence': 'Send these guys back!',
  'token': 1669,
  'token_str': ' guys'},
 {'score': 0.0757276862859726,
  'sequence': 'Send these women back!',
  'token': 390,
  'token_str': ' women'},
 {'score': 0.07122790068387985,
  'sequence': 'Send these girls back!',
  'token': 1972,
  'token_str': ' girls'}]

[Reference](https://towardsdatascience.com/transformers-retraining-roberta-base-using-the-roberta-mlm-procedure-7422160d5764)